In [ ]:
#Import metadata
#metadata = pd.read_csv(metaPath, sep=",")

metadata['global_contrast'] = metadata['tbp_lv_deltaA'] + metadata['tbp_lv_deltaB'] + metadata['tbp_lv_deltaL']
metadata['normalized_contrast'] = (metadata['tbp_lv_deltaA'] + metadata['tbp_lv_deltaB'] + metadata['tbp_lv_deltaL']) / 3
metadata['delta_E'] = ((metadata['tbp_lv_L'] - metadata['tbp_lv_Lext'])**2 + 
                      (metadata['tbp_lv_A'] - metadata['tbp_lv_Aext'])**2 + 
                      (metadata['tbp_lv_B'] - metadata['tbp_lv_Bext'])**2)**0.5
metadata['shape_irregularity'] = metadata['tbp_lv_eccentricity'] * metadata['tbp_lv_area_perim_ratio']
metadata['size_to_perimeter_ratio'] = metadata['clin_size_long_diam_mm'] / metadata['tbp_lv_perimeterMM']
metadata['color_asymmetry_ratio'] = metadata['tbp_lv_radial_color_std_max'] / metadata['tbp_lv_areaMM2']
metadata['size_to_area_ratio'] = metadata['clin_size_long_diam_mm'] / metadata['tbp_lv_areaMM2']
# luminance (light intensity) ratio between internal and  external lesion part
metadata['luminance_ratio'] = metadata['tbp_lv_L'] / metadata['tbp_lv_Lext']

# ChrominanceA (color intensity) ratio between internal and external lesion part
metadata['chroma_A_ratio'] = metadata['tbp_lv_A'] / metadata['tbp_lv_Aext']

# ChrominanceB (color intensity) 

In [ ]:
ratio between internal and external lesion part
metadata['chroma_B_ratio'] = metadata['tbp_lv_B'] / metadata['tbp_lv_Bext']


#METADATA: color and size features having no NAs
metadata = metadata[["isic_id",
                     "age_approx",
                     "target",
                     "clin_size_long_diam_mm",
                     "tbp_lv_areaMM2",
                     "tbp_lv_area_perim_ratio",
                     "tbp_lv_eccentricity",
                     "tbp_lv_minorAxisMM",
                     "tbp_lv_color_std_mean",
                     "tbp_lv_deltaLBnorm",
                     "tbp_lv_radial_color_std_max",
                     "tbp_lv_location",
                     'global_contrast',
                     'normalized_contrast',
                     'delta_E',
                     'shape_irregularity',
                     'size_to_perimeter_ratio',
                     'color_asymmetry_ratio',
                     'size_to_area_ratio',
                     'luminance_ratio',
                     'chroma_A_ratio',
                    'chroma_B_ratio']]



#Verify that there are no NAs
print("-- X_meta NA counts --")
print(metadata.isna().sum())

#Check number of Unknoxn for tbp_lv_location
loc_unknown=metadata[metadata["tbp_lv_location"]=="Unknown"]
print("Number of unknown for tbp_lv_location", len(loc_unknown))

In [ ]:
metadata=metadata[metadata["tbp_lv_location"]!="Unknown"]

loc_unknown2=metadata[metadata["tbp_lv_location"]=="Unknown"]
print("Number of unknown for tbp_lv_location", len(loc_unknown2))

In [ ]:
#Apply One-hot encoding for location
location=pd.get_dummies(metadata["tbp_lv_location"],prefix='category')
location = location.astype(int)
metadata = pd.concat([metadata, location], axis=1)
metadata=metadata.drop("tbp_lv_location",axis=1)
print(metadata.columns)

In [ ]:
# Calculate the mean of age_approx for each target group
mean_age_malign = metadata.loc[metadata["target"] == 1, "age_approx"].mean()
mean_age_benign = metadata.loc[metadata["target"] == 0, "age_approx"].mean()

# Define a function to fill NA based on the target value
def fill_na_by_target(row):
    if pd.isna(row['age_approx']):
        if row['target'] == 1:
            return mean_age_malign
        elif row['target'] == 0:
            return mean_age_benign
    return row['age_approx']

# Apply the function to the age_approx column
metadata['age_approx'] = metadata.apply(fill_na_by_target, axis=1)

#Verify that there are no NAs
print("-- X_meta NA counts --")
print(metadata.isna().sum())
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#Normalization
#Select the column

feature=metadata.drop(columns=['isic_id','target', 'category_Head & Neck',
       'category_Left Arm', 'category_Left Arm - Lower',
       'category_Left Arm - Upper', 'category_Left Leg',
       'category_Left Leg - Lower', 'category_Left Leg - Upper',
       'category_Right Arm', 'category_Right Arm - Lower',
       'category_Right Arm - Upper', 'category_Right Leg',
       'category_Right Leg - Lower', 'category_Right Leg - Upper',
       'category_Torso Back Bottom Third', 'category_Torso Back Middle Third',
       'category_Torso Back Top Third', 'category_Torso Front',
       'category_Torso Front Bottom Half', 'category_Torso Front Top Half'])


#scaler=StandardScaler() for standardization
scaler = MinMaxScaler()
feature_standardized=scaler.fit_transform(feature)
feature_standardized_df = pd.DataFrame(feature_standardized, columns=feature.columns)


metadata=pd.concat([metadata[['isic_id','target', 'category_Head & Neck',
       'category_Left Arm', 'category_Left Arm - Lower',
       'category_Left Arm - Upper', 'category_Left Leg',
       'category_Left Leg - Lower', 'category_Left Leg - Upper',
       'category_Right Arm', 'category_Right Arm - Lower',
       'category_Right Arm - Upper', 'category_Right Leg',
       'category_Right Leg - Lower', 'category_Right Leg - Upper',
       'category_Torso Back Bottom Third', 'category_Torso Back Middle Third',
       'category_Torso Back Top Third', 'category_Torso Front',
       'category_Torso Front Bottom Half', 'category_Torso Front Top Half']].reset_index(drop=True), feature_standardized_df] , axis=1)

print(len(metadata.columns))